In [14]:
import pandas as pd
import numpy as np
from IPython.display import display 
import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [12]:
# Uncomment and run the following line if Altair is not installed:
# pip install altair

     ------------------------------------ 731.2/731.2 kB 470.6 kB/s eta 0:00:00
     -------------------------------------- 419.6/419.6 kB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
data = pd.read_csv(r"Data\bank.csv", sep=';', dtype=str, encoding='latin-1')
print(f"Successfully loaded data: {data.shape[0]} rows, {data.shape[1]} columns")
print(f"Initial columns: {data.columns.tolist()}")

data_copy = data.copy()
display(data_copy.head(5))

data_copy.columns = data_copy.columns.str.lower().str.replace(' ', '_', regex=False).str.replace('[^0-9a-zA-Z_]', '', regex=True)

print("\nSample of cleaned data (first few rows):")
display(data_copy.head(5)) # Just used for debugging to check what the DataFrame look like 

print(f"\nData types after cleaning:\n{data_copy.dtypes}") # Show what the data types of the columnds are after cleaning 
print(f"\nFinal columns available for use: {data_copy.columns.tolist()}")

Successfully loaded data: 4521 rows, 17 columns
Initial columns: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']


ValueError: could not convert string to float: 'unemployed'